In [1]:
import pandas as pd
import mysql.connector

In [2]:
cell_phones_df = pd.read_csv("cell_phones_total.csv")
population_df = pd.read_csv("pop.csv")

In [5]:
cell_phones_transposed = cell_phones_df.T
population_transposed = population_df.T


In [6]:
print("Transposed Cell Phones Data:")
print(cell_phones_transposed)

Transposed Cell Phones Data:
           0            1       2        3        4      5          6    \
country  Aruba  Afghanistan  Angola  Albania  Andorra    UAE  Argentina   
1960       0.0          0.0     0.0      0.0      0.0    NaN        0.0   
1961       NaN          NaN     NaN      NaN      NaN    NaN        NaN   
1962       NaN          NaN     NaN      NaN      NaN    NaN        NaN   
1963       NaN          NaN     NaN      NaN      NaN    NaN        NaN   
...        ...          ...     ...      ...      ...    ...        ...   
2019      141k        22.6M   14.8M    2.63M    87.9k  19.6M      56.4M   
2020      141k        22.7M   14.6M    2.62M    80.4k  18.4M      54.8M   
2021      141k        22.7M   15.3M    2.64M    93.8k  18.2M      59.1M   
2022       NaN        22.8M   23.7M    2.78M     114k    20M      59.7M   
2023       NaN          23M   25.7M    2.61M     126k  21.2M      62.7M   

             7               8                    9    ...        204 

In [21]:
cell_phones_df.columns = cell_phones_df.Tcolumns.str.strip()
population_df.columns = population_df.Tcolumns.str.strip()


AttributeError: 'DataFrame' object has no attribute 'Tcolumns'

In [11]:
cell_phones_df = cell_phones_df.melt(id_vars=['country'], var_name='Year', value_name='Cell_Phone_Total')
population_df = population_df.melt(id_vars=['country'], var_name='Year', value_name='Population')


In [12]:
cell_phones_df.rename(columns={'country': 'Country'}, inplace=True)
population_df.rename(columns={'country': 'Country'}, inplace=True)
cell_phones_df['Year'] = cell_phones_df['Year'].astype(int)
population_df['Year'] = population_df['Year'].astype(int)


In [13]:
db = mysql.connector.connect(host="localhost",user="root",password="Logesh",database="bootcamp")
cursor = db.cursor()
print(db)
cursor = db.cursor()


In [14]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS CellPhones (
        Country VARCHAR(100),
        Year INT,
        Cell_Phone_Total BIGINT,
        PRIMARY KEY (Country, Year)
    )
""")


In [15]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Population (
        Country VARCHAR(100),
        Year INT,
        Population VARCHAR(20),  # VARCHAR to handle values like "3.28M"
        PRIMARY KEY (Country, Year)
    )
""")


In [16]:
db.commit()
print("Tables created successfully!")


Tables created successfully!


In [17]:
for _, row in cell_phones_df.iterrows():
    cell_phone_total = row['Cell_Phone_Total'] if pd.notna(row['Cell_Phone_Total']) else 0
    cursor.execute(
        "INSERT IGNORE INTO CellPhones (Country, Year, Cell_Phone_Total) VALUES (%s, %s, %s)",
        (row['Country'], row['Year'], cell_phone_total)
    )

for _, row in population_df.iterrows():
    cursor.execute(
        "INSERT IGNORE INTO Population (Country, Year, Population) VALUES (%s, %s, %s)",
        (row['Country'], row['Year'], row['Population'])
    )



In [18]:
db.commit()
print("Data inserted successfully!")


Data inserted successfully!
